In [0]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


In [0]:
# load ascii text and covert to lowercase
filename = "alice.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

In [0]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [0]:
  n_chars = len(raw_text)
  n_vocab = len(chars)
  print("Total Characters: ", n_chars)
  print("Total Vocab: ", n_vocab)

Total Characters:  144139
Total Vocab:  44


In [0]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
# range(start_vaue, end_value, step)
for i in range(0, n_chars - seq_length, 1):  
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  144039


In [0]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [0]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]),return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
#model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

W0713 04:53:46.558779 140385624586112 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0713 04:53:46.609258 140385624586112 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0713 04:53:46.622010 140385624586112 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0713 04:53:47.001658 140385624586112 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0713 04:53:47.015373 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100, 256)          264192    
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 256)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 43)                11051     
Total params: 800,555
Trainable params: 800,555
Non-trainable params: 0
_________________________________________________________________


In [0]:
# define the checkpoint
filepath="/content/drive/My Drive/Colab_Models/alice_in_wonderland_improved_6.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [0]:
model.fit(X, y, epochs=10, batch_size=256, callbacks=callbacks_list)

W0712 06:39:34.516344 140282880669568 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/10
144039/144039 [==============================] - 168s 1ms/step - loss: 1.0768

Epoch 00001: loss improved from inf to 1.07678, saving model to /content/drive/My Drive/Colab_Models/alice_in_wonderland_improved_6.hdf5
Epoch 2/10
144039/144039 [==============================] - 166s 1ms/step - loss: 1.0705

Epoch 00002: loss improved from 1.07678 to 1.07054, saving model to /content/drive/My Drive/Colab_Models/alice_in_wonderland_improved_6.hdf5
Epoch 3/10
144039/144039 [==============================] - 165s 1ms/step - loss: 1.0725

Epoch 00003: loss did not improve from 1.07054
Epoch 4/10
144039/144039 [==============================] - 166s 1ms/step - loss: 1.0655

Epoch 00004: loss improved from 1.07054 to 1.06549, saving model to /content/drive/My Drive/Colab_Models/alice_in_wonderland_improved_6.hdf5
Epoch 5/10
144039/144039 [==============================] - 167s 1ms/step - loss: 1.0689

Epoch 00005: loss did not improve from 1.06549
Epoch 6/10
144039/144039 [===========

In [0]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [0]:
import sys
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("\n=========================================Seed Text======================================================\n")
str = ""
print(str.join([int_to_char[value] for value in pattern]))
print("\n========================================Predicted Text=================================================== \n")

# generate characters
for i in range(1000):
  x = numpy.reshape(pattern, (1, len(pattern), 1))
  x = x / float(n_vocab)
  prediction = model.predict(x, verbose=0)
  index = numpy.argmax(prediction)
  result = int_to_char[index]
  seq_in = [int_to_char[value] for value in pattern]
  sys.stdout.write(result)
  if result == "," or result == ".":
    sys.stdout.write("\n")
  pattern.append(index)
  pattern = pattern[1:len(pattern)]

print("\n\nDone.")


=========================================Seed Text======================================================

m mad.'

'i call it purring, not growling,' said alice.

'call it what you like,' said the cat. 'do 

========================================Predicted Text=================================================== 

you teal to thin before?'

'i don't know what you cone to she tone,
' said the ming an snop alice,
 and she thought it ovt think of the mock turtle was a little before,
 but the mock turtle said to the mock turtle,
 so she tere all that she had not artended the words: 'what a catcus-race?' 
'i don't know what you cone to she tone,
' said the ming an snop alice,
 and she thought it ovt think of the mock turtle was a little before,
 but the mock turtle said to the mock turtle,
 so she tere all that she had not artended the words: 'what a catcus-race?' 
'i don't know what you cone to she tone,
' said the ming an snop alice,
 and she thought it ovt think of the mock turtle was a

In [0]:
# batch 1
# epochs 10 batch size = 128
# batch 2
# epochs 10 batch_size = 64
# batch 3
# epochs 10 batch_size = 64 filename = improved_3
# batch 4
# epochs 20 batch_size = 256 filename = improved_4 loss = 1.114
# batch 5
# epochs 10 batch_size = 256 filename = improved_5 loss = 1.0806
# batch 6
# epochs 10 batch_size = 256 filename = improved_6 loss = 1.05